https://stackoverflow.com/questions/64156202/add-dense-layer-on-top-of-huggingface-bert-model

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Install requirements

In [1]:
# !pip install Datasets
# !pip install transformers
# !pip install scikit-learn

  Using cached datasets-2.16.1-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.15-py311-none-any.whl.metadata (7.2 kB)
  Using cached huggingface_hub-0.20.2-py3-none-any.whl.metadata (12 kB)
Using cached datasets-2.16.1-py3-none-any.whl (507 kB)
Using cached dill-0.3.7-py3-none-any.whl (115 kB)
Using cached huggingface_hub-0.20.2-py3-none-any.whl (330 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Using cached multiprocess-0.70.15-py311-none-any.whl (135 kB)
Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
Using cached xxhash-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Using cached transformers-4.36.2-py3-none-any.whl.metadata (12

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
import tensorflow as tf
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import SparseCategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import datasets
import keras

2024-01-14 08:35:44.260501: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-14 08:35:44.304514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 08:35:44.304547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 08:35:44.305732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 08:35:44.313179: I tensorflow/core/platform/cpu_feature_guar

## Define custom model

Using a cased version of BERT will mean a bigger vocabulary, which in our case will slow down the training and use up more memory.
Some tricks that may (or may not) improve training time: **Mixed precision** and **jit_compile**.

In [26]:
# # try to improve the learning time
# from tensorflow.keras import mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


2024-01-13 10:30:08.749258: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [2]:
def scheduler(epoch, lr):
  if epoch == 10:
    return lr / 2
  else:
    return lr


In [3]:
MODEL_ID = "bert-base-uncased" # define here the model to use

In [4]:
class CustomBert(keras.Model):
    """Appending additional layers to the BERT model, so that it can be used
    for multi-label classification.
    """

    def __init__(self, base_model_id: str, num_labels: int):
        super().__init__()
        self._base = TFAutoModel.from_pretrained(base_model_id)
        self._base.trainable = False

        self._additional_layers = keras.Sequential([
            Dropout(0.1),
            Dense(512, activation="relu"),
            Dense(256, activation="relu"),
            Dense(num_labels, activation="sigmoid"),
        ])

    def call(self, inputs):
        out = self._base(inputs)
        out = out["last_hidden_state"][:, 0, :]

        return self._additional_layers(out)

### Use for debug only

In [8]:
# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

In [5]:
def preprocess_function(examples, tokenizer, max_length=128):
    return tokenizer(examples["text"],
                     truncation=True,
                     padding='max_length',
                     max_length=max_length,
                     return_tensors="np")

In [6]:
def get_test_data(test_path):
    test_df = pd.read_json(test_path, lines=True)
    test_df = test_df[['text', 'label']]
    return test_df

In [7]:
def get_data(train_path):
   train_df = pd.read_json(train_path, lines=True)
   train_df = train_df[['text', 'label']]
   train_df, validation_df = train_test_split(train_df, test_size=0.1, stratify=train_df['label'])
   return train_df, validation_df


In [8]:
file_train_path = '../data/subtaskA_train_monolingual.jsonl'

In [9]:
train_df, validation_df = get_data(file_train_path)

In [10]:
file_test_path = '../data/subtaskA_dev_monolingual.jsonl'

In [11]:
test_df = get_test_data(file_test_path)

In [12]:
print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {validation_df.shape}")
print(f"Test shape: {test_df.shape}")

Train shape: (107781, 2)
Validation shape: (11976, 2)
Test shape: (5000, 2)


In [17]:
# take only 100 sample from training data (test purposes)
# train_df = train_df[:100]
# validation_df = validation_df[:100]

In [13]:
id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1 }

In [14]:
# pandas dataframe to huggingface Dataset
train_dataset= Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

In [20]:
print(train_dataset)

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 107781
})


In [15]:
dataset = datasets.DatasetDict({'train': train_dataset, 'validation': validation_dataset, 'test' : test_dataset})

In [22]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 107781
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 11976
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})


In [23]:
dataset["train"][0]

{'text': '\n\nThis paper addresses the problem of segmentation and labeling of high dimensional time series data in an unsupervised way. The authors propose a Recurrent Hidden Semi-Markov Model (R-HSMM) to tackle the limitation of existing HSMM models with simple conditional assumptions of observations. Strengths of the paper include the design of a structure encoding function to accelerate the inference while preserving accuracy, and the generalization of the penalty method to distribution space to enable simultaneous training of the model and encoding function. Weaknesses include the lack of a comparison between the proposed R-HSMM and other existing methods, and the lack of a discussion of the computational complexity of the proposed model.',
 'label': 1,
 '__index_level_0__': 45339}

In [16]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [21]:
# pre_tokenizer_columns = set(dataset["train"].features)
encoded_dataset = dataset.map(preprocess_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
# tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
# print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/107781 [00:00<?, ? examples/s]

Map:   0%|          | 0/11976 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [22]:
model_bert = CustomBert(MODEL_ID, 1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [23]:
prepare_model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=len(label2id), id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
tf_train_dataset = prepare_model.prepare_tf_dataset(encoded_dataset["train"],
                                      batch_size=64,
                                      shuffle=True,
                                      tokenizer=tokenizer)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
tf_validation_dataset = prepare_model.prepare_tf_dataset(encoded_dataset["validation"],
                                          batch_size=64,
                                          shuffle=True,
                                          tokenizer=tokenizer)

In [26]:
tf_test_dataset = prepare_model.prepare_tf_dataset(encoded_dataset["test"],
                                          batch_size=64,
                                          shuffle=True,
                                          tokenizer=tokenizer)

In [28]:
del prepare_model

In [32]:
optimizer = keras.optimizers.Adam(learning_rate=2e-4) 
# loss_fn = SparseCategoricalCrossentropy(from_logits=False)
loss_fn = BinaryCrossentropy(from_logits=False) # use with activation = sigmoid for last layer
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [27]:
# model.compile(optimizer=optimizer, loss=loss_fn, metrics=[train_acc_metric])
model_bert.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'], jit_compile=True)

In [34]:
def train_step(model, tf_train_dataset, tf_test_dataset, epochs=2):
  # train the model by using GradientTape
  optimizer = keras.optimizers.Adam(learning_rate=5e-5)

  for epoch in range(epochs):
      print(f"\nStart of Training Epoch {epoch}")
      for step, batch in enumerate(tf_train_dataset):
          ids = batch[0]['input_ids']
          mask = batch[0]['attention_mask']
          y = batch[1]

          with tf.GradientTape() as tape:
              logits = model((ids, mask), training=True)
              loss_value = loss_fn(y, logits)

          grads = tape.gradient(loss_value, model.trainable_weights)

          optimizer.apply_gradients(
                        (grad, var)
                        for (grad, var) in zip(grads, model.trainable_variables)
                        if grad is not None
                      )
          # Update training metric.
          train_acc_metric(y, logits)

          # Log every 200 batches.
          if step % 10 == 0:
              print(
                  "Training loss at step %d: %.4f"
                  % (step, float(loss_value))
              )
              #print accuracy on the training set
              train_acc = train_acc_metric.result()
              print("Training acc over epoch: %.4f" % (float(train_acc),))
          # Display metrics at the end of each epoch.

      train_acc_metric.reset_states()

      # perform validation on test data
      for step, batch in enumerate(tf_test_dataset):
          ids = batch[0]['input_ids']
          mask = batch[0]['attention_mask']
          y = batch[1]
          logits = model([ids, mask], training=False)
          # Update val metrics
          val_acc_metric(y, logits)

      val_acc = val_acc_metric.result()

      # print accuracy on the test set
      print("Test acc: %.4f" % (float(val_acc),))

      # Reset val metrics at the end of each epoch
      val_acc_metric.reset_states()

  model.save_weights('my_model', save_format='tf')

In [28]:
model_bert.fit(tf_train_dataset,
               validation_data= tf_test_dataset,
               epochs=15)

Epoch 1/15


2024-01-13 12:37:11.272255: I external/local_xla/xla/service/service.cc:168] XLA service 0x5587380d3b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-13 12:37:11.272299: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-01-13 12:37:11.621594: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:59] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. custom_bert_1/sequential_1/dropout_113/dropout/random_uniform/RandomUniform
2024-01-13 12:37:11.699462: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-13 12:37:11.756278: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator custom_bert_1/tf_bert_model_1/bert/embeddings/assert_less

1684/1684 [==============================] - ETA: 0s - loss: 0.4072 - accuracy: 0.8094

2024-01-13 12:48:11.576836: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator custom_bert_1/tf_bert_model_1/bert/embeddings/assert_less/Assert/Assert


1684/1684 [==============================] - 700s 405ms/step - loss: 0.4072 - accuracy: 0.8094 - val_loss: 0.7122 - val_accuracy: 0.6715
Epoch 2/15
1684/1684 [==============================] - 678s 403ms/step - loss: 0.3509 - accuracy: 0.8403 - val_loss: 0.6637 - val_accuracy: 0.6825
Epoch 3/15
1684/1684 [==============================] - 678s 403ms/step - loss: 0.3314 - accuracy: 0.8514 - val_loss: 0.6545 - val_accuracy: 0.7027
Epoch 4/15
1684/1684 [==============================] - 678s 403ms/step - loss: 0.3203 - accuracy: 0.8555 - val_loss: 0.6299 - val_accuracy: 0.7103
Epoch 5/15
1684/1684 [==============================] - 678s 403ms/step - loss: 0.3108 - accuracy: 0.8606 - val_loss: 0.6133 - val_accuracy: 0.7254
Epoch 6/15
1684/1684 [==============================] - 678s 403ms/step - loss: 0.3058 - accuracy: 0.8633 - val_loss: 0.6332 - val_accuracy: 0.7206
Epoch 7/15
1684/1684 [==============================] - 678s 403ms/step - loss: 0.2977 - accuracy: 0.8676 - val_loss: 0.664

In [ ]:
## Analyze model arhitecture

In [29]:
model_bert.summary()

Model: "custom_bert_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model_1 (TFBertMod  multiple                  109482240 
 el)                                                             
                                                                 
 sequential_1 (Sequential)   (64, 1)                   525313    
                                                                 
Total params: 110007553 (419.65 MB)
Trainable params: 525313 (2.00 MB)
Non-trainable params: 109482240 (417.64 MB)
_________________________________________________________________


## Total layers of Bert model used

In [30]:
total_layers = len(model_bert._base.bert.encoder.layer)

In [31]:
print(total_layers)

12


In [ ]:
## Unfreeze some layers from Bert to improve accuracy

In [32]:
model_bert._base.trainable = True

In [33]:
for i, layer in enumerate(model_bert._base.bert.encoder.layer):
    if i < total_layers - 1:
        layer.trainable = False
    else:
        layer.trainable = True

In [34]:
# Use legacy optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5)  # Adjust the learning rate as needed


In [35]:
model_bert.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'], jit_compile=True)

## vizualize the number of trainable param after unfreeze

In [36]:
model_bert.summary()

Model: "custom_bert_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model_1 (TFBertMod  multiple                  109482240 
 el)                                                             
                                                                 
 sequential_1 (Sequential)   (64, 1)                   525313    
                                                                 
Total params: 110007553 (419.65 MB)
Trainable params: 32040961 (122.23 MB)
Non-trainable params: 77966592 (297.42 MB)
_________________________________________________________________


## Train again the model

In [38]:
model_bert.fit(tf_train_dataset,
               validation_data= tf_test_dataset,
               epochs=8)

Epoch 1/8
1684/1684 [==============================] - ETA: 0s - loss: 0.2248 - accuracy: 0.9031

2024-01-13 15:54:29.037568: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator custom_bert_1/tf_bert_model_1/bert/embeddings/assert_less/Assert/Assert


1684/1684 [==============================] - 1604s 952ms/step - loss: 0.2248 - accuracy: 0.9031 - val_loss: 0.6746 - val_accuracy: 0.7276
Epoch 2/8
1684/1684 [==============================] - 1602s 951ms/step - loss: 0.1883 - accuracy: 0.9206 - val_loss: 0.7219 - val_accuracy: 0.7252
Epoch 3/8
1684/1684 [==============================] - 1601s 951ms/step - loss: 0.1677 - accuracy: 0.9297 - val_loss: 0.7747 - val_accuracy: 0.7250
Epoch 4/8
1684/1684 [==============================] - 1601s 951ms/step - loss: 0.1499 - accuracy: 0.9386 - val_loss: 0.7299 - val_accuracy: 0.7398
Epoch 5/8
1684/1684 [==============================] - 1601s 951ms/step - loss: 0.1329 - accuracy: 0.9457 - val_loss: 0.7538 - val_accuracy: 0.7428
Epoch 6/8
1684/1684 [==============================] - 1601s 951ms/step - loss: 0.1182 - accuracy: 0.9523 - val_loss: 0.8632 - val_accuracy: 0.7316
Epoch 7/8
1684/1684 [==============================] - 1600s 950ms/step - loss: 0.1064 - accuracy: 0.9577 - val_loss: 0.83

In [ ]:
# predict on test data
predictions = model_bert.predict(tf_validation_dataset)

In [ ]:
model_bert.evaluate(tf_validation_dataset)

In [ ]:
print(predictions[0])

### Save Model

In [40]:
# The recommended way to save a subclassed model is to use save_weights to create a TensorFlow SavedModel checkpoint
model_bert.save_weights('my_model', save_format='tf')

### Load Model

In [28]:
loaded_model = CustomBert(MODEL_ID, 1)
loaded_model.load_weights('my_model')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [33]:
# Use legacy optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5)  # Adjust the learning rate as needed
loaded_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'], jit_compile=True)

In [34]:
loaded_model.evaluate(tf_test_dataset)

2024-01-14 08:41:49.637717: I external/local_xla/xla/service/service.cc:168] XLA service 0x5557f2881e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-14 08:41:49.637761: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-01-14 08:41:49.793678: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator custom_bert_2/tf_bert_model_2/bert/embeddings/assert_less/Assert/Assert
2024-01-14 08:41:49.797835: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-14 08:41:50.610975: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1705221713.046352     302 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


78/78 [==============================] - 40s 396ms/step - loss: 0.9186 - accuracy: 0.7458


[0.9186090230941772, 0.7457932829856873]

### Train the loaded model on a batch

In [ ]:
# Extract a batch from the training dataset
batch = next(iter(tf_train_dataset))

# Separate input data and target data from the batch
ids = batch[0]['input_ids']
mask = batch[0]['attention_mask']
y = batch[1]

In [ ]:
# call the model on the extracted batch
loss_value = loaded_model.train_on_batch((ids, mask), y)
print(f"Loss: {loss_value}")

### Check that the model has been preserved

In [ ]:
loaded_model.load_weights('my_model')

In [ ]:
val_acc_metric.reset_state()
for step, batch in enumerate(tf_test_dataset):
        ids = batch[0]['input_ids']
        mask = batch[0]['attention_mask']
        y = batch[1]
        logits = loaded_model([ids, mask], training=False)
        # Update val metrics
        val_acc_metric(y, logits)
val_acc = val_acc_metric.result()
# print accuracy on the test set
print("test acc: %.4f" % (float(val_acc),))

In [ ]:
# Check that the model state has been preserved
new_predictions = loaded_model.predict(tf_test_dataset)
# np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)


### Another save

In [ ]:
# save the custom model to file (not suported for custom class)
model_bert.save('path_to_my_model',save_format='tf')

In [ ]:
# load the model
new_model = keras.models.load_model('path_to_my_model')